In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
images = list(data_dir.rglob('*.jpg'))

In [ ]:
images_count = len(images)

In [ ]:
PIL.Image.open(str(images[46]))

In [ ]:
flowers_images_dict = {
    'roses': list(data_dir.rglob('roses/*')),
    'daisy': list(data_dir.rglob('daisy/*')),
    'dandelion': list(data_dir.rglob('dandelion/*')),
    'sunflowers': list(data_dir.rglob('sunflowers/*')),
    'tulips': list(data_dir.rglob('tulips/*')),
}

In [ ]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [ ]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))
img

In [ ]:
img.shape

In [ ]:
cv2.resize(img,(180,180)).shape

In [ ]:
X,y = [],[]
for img_name , imgs in flowers_images_dict.items():
    for img in imgs:
        image = cv2.imread(img)
        image_resize = cv2.resize(image,(224,224))
        X.append(image_resize)
        y.append(flowers_labels_dict[img_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
len(X_train) , len(X_test)

In [ ]:
X_train.shape , X_test.shape

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [ ]:
model = keras.Sequential([
    keras.layers.RandomFlip("horizontal", input_shape=(224,224,3)),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),

    keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),

    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(len(flowers_labels_dict), activation='softmax')
])

In [ ]:
model.compile(
    optimizer = 'adam' ,
    loss = 'sparse_categorical_crossentropy' ,
    metrics = ['accuracy']
    )

In [ ]:
model.fit(X_train_scaled,y_train,epochs=10)

In [ ]:
model.evaluate(X_test_scaled,y_test)